In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import process

In [2]:
# Import dataset
df = pd.read_csv('data/crash_reporting_drivers_data.csv')
df.head()

/var/folders/wh/_bf85_z53cl0mz04lchklxx00000gn/T/ipykernel_72756/879525709.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/crash_reporting_drivers_data.csv')


,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,...,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location
0,MCP3040003N,190026050,Montgomery County Police,Property Damage Crash,05/31/2019 03:00:00 PM,NaN,NaN,NaN,NaN,PARKING LOT OF 3215 SPARTAN RD,...,15,No,No,2004,HONDA,TK,UNKNOWN,39.150044,-77.063089,"(39.15004368, -77.06308884)"
1,MCP1307000K,190024786,Montgomery County Police,Property Damage Crash,05/24/2019 05:00:00 PM,NaN,NaN,NaN,NaN,PARKING LOT,...,0,No,No,0,UNK,UNK,NaN,39.199047,-77.250743,"(39.19904667, -77.25074333)"
2,MCP2846008X,230034260,Montgomery County Police,Property Damage Crash,07/17/2023 10:45:00 AM,County,SELFRIDGE RD,County,RANDOLPH RD,NaN,...,35,No,No,2003,FORD,TK,NO MISUSE,39.054588,-77.085974,"(39.05458848, -77.08597423)"
3,MCP32610017,230034668,Montgomery County Police,Property Damage Crash,07/20/2023 11:40:00 PM,Maryland (State),MUNCASTER MILL RD,County,SHADY GROVE RD,NaN,...,45,No,No,2023,TOYT,CP,NO MISUSE,39.148721,-77.147111,"(39.14872076, -77.14711061)"
4,EJ78520081,230033429,Gaithersburg Police Depar,Property Damage Crash,07/13/2023 05:40:00 PM,Municipality,PERRY PKWY,Unknown,ENT TO SHOPPING CENTER,NaN,...,25,No,No,2001,KENWORTH,TRUCK,NaN,39.149085,-77.210731,"(39.14908542, -77.21073135)"


In [3]:
# Print all the 43 column names
df.columns

Index(['Report Number', 'Local Case Number', 'Agency Name', 'ACRS Report Type',
       'Crash Date/Time', 'Route Type', 'Road Name', 'Cross-Street Type',
       'Cross-Street Name', 'Off-Road Description', 'Municipality',
       'Related Non-Motorist', 'Collision Type', 'Weather',
       'Surface Condition', 'Light', 'Traffic Control',
       'Driver Substance Abuse', 'Non-Motorist Substance Abuse', 'Person ID',
       'Driver At Fault', 'Injury Severity', 'Circumstance',
       'Driver Distracted By', 'Drivers License State', 'Vehicle ID',
       'Vehicle Damage Extent', 'Vehicle First Impact Location',
       'Vehicle Second Impact Location', 'Vehicle Body Type',
       'Vehicle Movement', 'Vehicle Continuing Dir', 'Vehicle Going Dir',
       'Speed Limit', 'Driverless Vehicle', 'Parked Vehicle', 'Vehicle Year',
       'Vehicle Make', 'Vehicle Model', 'Equipment Problems', 'Latitude',
       'Longitude', 'Location'],
      dtype='object')

In [4]:
# Function to print the unique values and their frequency in each column in tablular format also prints the number of NA values in each column
def print_unique_values(df):
    for col in df.columns:
        info_series = df[col].value_counts()
        print(info_series)
        print('NA values: ', df[col].isna().sum())
        print('Max freq: ', max(info_series))
        print('Min freq: ', min(info_series))
        print('len: ', len(info_series))
        print('\n__________________________\n')
print_unique_values(df)

Report Number
MCP229800RC    10
MCP2693003Q     9
MCP12130045     9
MCP28810040     8
MCP2667000H     8
               ..
MCP12130055     1
MCP33200017     1
MCP29620050     1
MCP2361001B     1
MCP3090006L     1
Name: count, Length: 94388, dtype: int64
NA values:  0
Max freq:  10
Min freq:  1
len:  94388

__________________________

Local Case Number
200022715    9
180051879    9
190009085    7
220055904    7
16046398     7
            ..
16027040     1
16028341     1
15000578     1
16021525     1
220047494    1
Name: count, Length: 115683, dtype: int64
NA values:  0
Max freq:  9
Min freq:  1
len:  115683

__________________________

Agency Name
Montgomery County Police     134943
MONTGOMERY                    10659
Rockville Police Departme      9514
Gaithersburg Police Depar      7430
Takoma Park Police Depart      2877
Maryland-National Capital      1122
ROCKVILLE                       630
GAITHERSBURG                    390
TAKOMA                          157
MCPARK                

## Pre-Processing

In [5]:
df = pd.read_csv('data/crash_reporting_drivers_data.csv')

df['Crash Date/Time'] = pd.to_datetime(df['Crash Date/Time'], format='%m/%d/%Y %I:%M:%S %p')

# remove_leading_trailing_whitespaces
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.strip()

# Removing unwanted columns
df.drop(["Report Number", "Local Case Number", "Person ID", "Drivers License State", "Vehicle ID", "Speed Limit", "Driverless Vehicle", "Parked Vehicle"], inplace=True, axis=1)


/var/folders/wh/_bf85_z53cl0mz04lchklxx00000gn/T/ipykernel_72756/797135475.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/crash_reporting_drivers_data.csv')


### Utilities

In [6]:
def display_unique_values(df,col, max=True):
    if col not in df.columns:
        print(f"{col} not in dataframe")
        return
    if not max:
        print(df[col].value_counts())
    else:
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            print(df[col].value_counts())

    print(f"\nLength of Unique Values: {len(df[col].value_counts())}")
    print(f"\nColumn Length: {len(df[col])}")
    print('NA values: ', df[col].isna().sum())

#### Vehicle Year

In [7]:
# Vehicle Year: sanitize the data and convert to datetime
display_unique_values(df, 'Vehicle Year')
df['Vehicle Year'] = df['Vehicle Year'].astype(str)

def update_vehicle_year(value):
    if len(value) == 1:
        return '200' + value
    elif len(value) == 2 and value.startswith('20'):
        return '20' + value
    elif len(value) == 2 and value.startswith('99'):
        return '1999'
    elif len(value) == 3 and value.startswith('2'):
        return '20' + value[-2]
    elif len(value) == 3 and value.startswith('9'):
        return '1' + value
    else:
        return value

df['Vehicle Year'] = df['Vehicle Year'].apply(update_vehicle_year)
df['Vehicle Year'] = df['Vehicle Year'].astype(int)
for index, row in df.iterrows():
    if row['Vehicle Year'] > 2023 or row['Vehicle Year'] <= 1900:
        df.at[index, 'Vehicle Year'] = 1800

print("\n__________________________\n")
df['Vehicle Year'] = df['Vehicle Year'].astype(str)
df['Vehicle Year'] = pd.to_datetime(df['Vehicle Year'], format='%Y')
display_unique_values(df, 'Vehicle Year')

Vehicle Year
2015    11573
2014    11406
2016    10901
2013    10704
2012     9154
2017     8788
2011     8229
2007     7808
2008     7759
2006     7309
2010     7290
2009     6661
2018     6473
2005     6419
2004     6079
2003     5294
2019     5188
2002     4225
0        3875
2001     3449
2020     3255
2000     3227
2021     2369
1999     2164
1998     1619
2022     1448
1997     1271
1996      800
1995      617
2023      507
1994      436
1993      300
1992      196
1991      154
1990      147
1989      110
1988       90
1987       64
1900       63
9999       59
1986       38
1985       33
99         26
2024       25
1983       15
1984       12
1978       10
1982        9
1980        9
1969        5
1974        5
1972        4
1111        4
1968        3
15          3
1979        3
2996        3
2914        3
1901        3
1981        3
1975        3
1966        3
2033        3
1970        3
1955        2
1977        2
1014        2
2912        2
1976        2
2104        2
1234   

### Vehicle Make

In [8]:
# Vehicle Make: Replace values with the most similar company name
display_unique_values(df, 'Vehicle Make')
print(len(df['Vehicle Make'].value_counts()))

# Actual company names
companies = ['FORD', 'LEXUS', 'TOYOTA', 'HYUNDAI', 'CHEVROLET', 'HONDA', 'NISSAN', 'DODGE', 'JEEP', 'KIA', 'BMW', 'MAZDA', 'SUBARU', 'GMC', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'ACURA', 'AUDI', 'CADILLAC', 'INFINITI', 'MITSUBISHI', 'CHRYSLER', 'BUICK', 'VOLVO', 'PONTIAC', 'LINCOLN', 'SATURN', 'MINI', 'SCION', 'PORSCHE', 'FIAT', 'MERCURY', 'SUZUKI', 'SMART', 'JAGUAR', 'LAND ROVER', 'SAAB', 'HUMMER', 'FERRARI', 'BENTLEY', 'ASTON MARTIN', 'ROLLS-ROYCE', 'LAMBORGHINI', 'MASERATI', 'ALFA ROMEO', 'DAEWOO', 'ISUZU', 'PORSC', 'HINO', 'FREIGHTLINER', 'KENWORTH', 'UD', 'MACK', 'STERLING', 'BLUE BIRD', 'THOMAS', 'INTERNATIONAL', 'WHITE', 'AMERICAN MOTORS', 'DAIHATSU', 'EAGLE', 'GEO', 'LOTUS', 'MAYBACH', 'MCLAREN', 'YAMAHA', 'TRIUMPH']

# Improvement: 1844 -> 931 unique values
replacement_dict = {}
replaced_counts = 0
for value in df['Vehicle Make'].unique():
    try:
        match = process.extractOne(value, companies)
        if match[1] >= 70:
            replacement_dict[value] = match[0]
            # print(f"{value} => {match[0]}")
            replaced_counts += 1
    except Exception as e:
        # print(f"Exception: {value}")
        continue
print(f"Replaced {replaced_counts} values")

df['Vehicle Make'] = df['Vehicle Make'].replace(replacement_dict)
df['Vehicle Make'].fillna('N/A', inplace=True)
print("\n__________________________\n")
display_unique_values(df, 'Vehicle Make')

Vehicle Make
TOYOTA                  22613
HONDA                   18447
FORD                    16741
TOYT                     8556
NISSAN                   8317
HOND                     5584
DODGE                    4414
HYUNDAI                  3815
CHEVROLET                3724
JEEP                     3679
UNKNOWN                  3594
CHEV                     3585
BMW                      3221
CHEVY                    3144
ACURA                    2781
LEXUS                    2750
KIA                      2611
NISS                     2580
SUBARU                   2141
MAZDA                    2097
HYUN                     2005
GMC                      1841
MERZ                     1664
THOMAS                   1517
MERCEDES                 1338
AUDI                     1319
GILL                     1033
VOLK                     1025
ACUR                      938
CHRYSLER                  928
SUBA                      903
VOLVO                     891
DODG                      8

Replaced 968 values

__________________________

Vehicle Make
TOYOTA                  31928
HONDA                   24112
FORD                    16784
NISSAN                  11109
CHEVROLET               10714
HYUNDAI                  6234
DODGE                    5297
ACURA                    4160
LEXUS                    3716
JEEP                     3692
UNKNOWN                  3594
BMW                      3225
SUBARU                   3195
VOLKSWAGEN               2954
MAZDA                    2853
KIA                      2612
THOMAS                   2413
MERCEDES-BENZ            2157
GMC                      1885
HUMMER                   1732
CHRYSLER                 1645
AUDI                     1324
INFINITI                 1240
VOLVO                    1237
GILL                     1033
MITSUBISHI               1010
CADILLAC                  938
BUICK                     922
NEW FLYER                 726
FRHT                      659
FREIGHTLINER              647
GILLIG  

/var/folders/wh/_bf85_z53cl0mz04lchklxx00000gn/T/ipykernel_72756/3206472879.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Vehicle Make'].fillna('N/A', inplace=True)


### Equipment Problems

In [9]:
# Equipment Problems: Replace NA values with a string "N/A"

display_unique_values(df, 'Equipment Problems')
df['Equipment Problems'] = df['Equipment Problems'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Equipment Problems')

Equipment Problems
NO MISUSE              120236
UNKNOWN                 13756
AIR BAG FAILED            167
OTHER                     108
BELT(S) MISUSED            33
BELTS/ANCHORS BROKE        12
STRAP/TETHER LOOSE          8
FACING WRONG WAY            6
NOT STREPPED RIGHT          3
SIZE/TYPE IMPROPER          2
Name: count, dtype: int64

Length of Unique Values: 10

Column Length: 167799
NA values:  33468

__________________________

Equipment Problems
NO MISUSE              120236
N/A                     33468
UNKNOWN                 13756
AIR BAG FAILED            167
OTHER                     108
BELT(S) MISUSED            33
BELTS/ANCHORS BROKE        12
STRAP/TETHER LOOSE          8
FACING WRONG WAY            6
NOT STREPPED RIGHT          3
SIZE/TYPE IMPROPER          2
Name: count, dtype: int64

Length of Unique Values: 11

Column Length: 167799
NA values:  0


### Vehicle Continuing Dir

In [10]:
# Vehicle Continuing Direction: Replace NA values with a string "N/A"
display_unique_values(df, 'Vehicle Continuing Dir')
df['Vehicle Continuing Dir'] = df['Vehicle Continuing Dir'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Vehicle Continuing Dir')

Vehicle Continuing Dir
North      45750
South      44715
East       35171
West       34034
Unknown     5521
Name: count, dtype: int64

Length of Unique Values: 5

Column Length: 167799
NA values:  2608

__________________________

Vehicle Continuing Dir
North      45750
South      44715
East       35171
West       34034
Unknown     5521
N/A         2608
Name: count, dtype: int64

Length of Unique Values: 6

Column Length: 167799
NA values:  0


### Vehicle Going Dir

In [11]:
# Vehicle Going Direction: Replace NA values with a string "N/A"
display_unique_values(df, 'Vehicle Going Dir')
df['Vehicle Going Dir'] = df['Vehicle Going Dir'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Vehicle Going Dir')

Vehicle Going Dir
North      46552
South      45420
East       34379
West       33922
Unknown     4918
Name: count, dtype: int64

Length of Unique Values: 5

Column Length: 167799
NA values:  2608

__________________________

Vehicle Going Dir
North      46552
South      45420
East       34379
West       33922
Unknown     4918
N/A         2608
Name: count, dtype: int64

Length of Unique Values: 6

Column Length: 167799
NA values:  0


### Vehicle Movement

In [12]:
# Vehicle Movement: Replace NA values with a string "N/A"
display_unique_values(df, 'Vehicle Movement')
df['Vehicle Movement'] = df['Vehicle Movement'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Vehicle Movement')

Vehicle Movement
MOVING CONSTANT SPEED      65426
SLOWING OR STOPPING        23978
STOPPED IN TRAFFIC LANE    18003
MAKING LEFT TURN           16138
ACCELERATING                9437
BACKING                     5655
MAKING RIGHT TURN           5268
CHANGING LANES              4551
STARTING FROM LANE          4402
UNKNOWN                     2735
PARKED                      2608
PARKING                     1895
STARTING FROM PARKED        1850
MAKING U TURN               1190
PASSING                      920
OTHER                        869
ENTERING TRAFFIC LANE        801
SKIDDING                     671
NEGOTIATING A CURVE          529
RIGHT TURN ON RED            260
LEAVING TRAFFIC LANE         205
DRIVERLESS MOVING VEH.        37
Name: count, dtype: int64

Length of Unique Values: 22

Column Length: 167799
NA values:  371

__________________________

Vehicle Movement
MOVING CONSTANT SPEED      65426
SLOWING OR STOPPING        23978
STOPPED IN TRAFFIC LANE    18003
MAKING LEFT TURN  

### Vehicle Body Type

In [13]:
# Vehicle Body Type: Replace NA values with a string "N/A" if the Vehicle Make is "UNKNOWN" or "N/A" and Vehicle Model "UNKNOWN" or "N/A"
display_unique_values(df, 'Vehicle Body Type')
df['Vehicle Body Type'] = df['Vehicle Body Type'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Vehicle Body Type')

# # Identify rows where 'Vehicle Make' and 'Vehicle Model' are 'UNKNOWN' or 'N/A'
# condition_unknown = (df['Vehicle Make'].isin(['UNKNOWN', 'N/A'])) & (df['Vehicle Model'].isin(['UNKNOWN', 'N/A']))

# # Identify rows where 'Vehicle Body Type' is NaN or 'N/A'
# condition_nan = pd.isna(df['Vehicle Body Type'])
# condition_na = df['Vehicle Body Type'] == 'N/A'

# # Counter calculations
# counter_1 = np.sum(condition_unknown)
# counter_2 = np.sum(condition_unknown & condition_nan)
# counter_3 = np.sum(condition_unknown & condition_na)
# counter_4 = len(df) - counter_1
# counter_5 = np.sum(~condition_unknown & condition_nan)
# counter_6 = np.sum(~condition_unknown & condition_na)

# # print all "Vehicle Body Type" values that satisfy the condition_unkown but do not satisfy condition_nan and condition_na
# # Identify rows that satisfy condition_unknown but not condition_nan and condition_na
# condition_specific = condition_unknown & ~condition_nan & ~condition_na

# # Print unique "Vehicle Body Type" values that satisfy the specific condition
# print("\n__________________________\n")
# print("Matching Vehicle Model and Make with Unique Values")
# print(df.loc[condition_specific, 'Vehicle Body Type'].unique())


# # Update 'Vehicle Body Type' where 'Vehicle Make' and 'Vehicle Model' are 'UNKNOWN' or 'N/A'
# df.loc[condition_unknown, 'Vehicle Body Type'] = 'N/A'

# # Display unique values of 'Vehicle Body Type'
# print("\n__________________________\n")
# display_unique_values(df, 'Vehicle Body Type')

# # Display counts
# print("\n__________________________\n")
# print(f"Matching Vehicle Model and Make: {counter_1}")
# print(f"Matching Vehicle Model and Make and Vehicle Body Type is NaN: {counter_2}")
# print(f"Matching Vehicle Model and Make and Vehicle Body Type is N/A: {counter_3}")
# print("\n__________________________\n")
# print(f"Different Vehicle Model and Make: {counter_4}")
# print(f"Different Vehicle Model and Make and Vehicle Body Type is NaN: {counter_5}")
# print(f"Different Vehicle Model and Make and Vehicle Body Type is N/A: {counter_6}")
# print("\n__________________________\n")
# print('NA values: ', df["Vehicle Body Type"].isna().sum())


Vehicle Body Type
PASSENGER CAR                                                116073
(SPORT) UTILITY VEHICLE                                       15575
PICKUP TRUCK                                                   6641
VAN                                                            4845
TRANSIT BUS                                                    3563
SCHOOL BUS                                                     2867
POLICE VEHICLE/NON EMERGENCY                                   2060
OTHER LIGHT TRUCKS (10,000LBS (4,536KG) OR LESS)               1874
CARGO VAN/LIGHT TRUCK 2 AXLES (OVER 10,000LBS (4,536 KG))      1815
OTHER                                                          1517
POLICE VEHICLE/EMERGENCY                                       1459
MEDIUM/HEAVY TRUCKS 3 AXLES (OVER 10,000LBS (4,536KG))         1442
UNKNOWN                                                        1024
MOTORCYCLE                                                      872
STATION WAGON                 

### Vehicle Second Impact Location

In [14]:
# Vehicle Second Impact Location: Replace NA values with a string "N/A"
display_unique_values(df, 'Vehicle Second Impact Location')
df['Vehicle Second Impact Location'] = df['Vehicle Second Impact Location'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Vehicle Second Impact Location')

Vehicle Second Impact Location
TWELVE OCLOCK    66048
SIX OCLOCK       33208
ONE OCLOCK       15474
ELEVEN OCLOCK    13109
TWO OCLOCK        6276
TEN OCLOCK        6114
SEVEN OCLOCK      4127
FOUR OCLOCK       4046
FIVE OCLOCK       3932
EIGHT OCLOCK      3691
THREE OCLOCK      3561
NINE OCLOCK       3457
UNKNOWN           2967
UNDERSIDE          636
ROOF TOP           530
NON-COLLISION      367
Name: count, dtype: int64

Length of Unique Values: 16

Column Length: 167799
NA values:  256

__________________________

Vehicle Second Impact Location
TWELVE OCLOCK    66048
SIX OCLOCK       33208
ONE OCLOCK       15474
ELEVEN OCLOCK    13109
TWO OCLOCK        6276
TEN OCLOCK        6114
SEVEN OCLOCK      4127
FOUR OCLOCK       4046
FIVE OCLOCK       3932
EIGHT OCLOCK      3691
THREE OCLOCK      3561
NINE OCLOCK       3457
UNKNOWN           2967
UNDERSIDE          636
ROOF TOP           530
NON-COLLISION      367
N/A                256
Name: count, dtype: int64

Length of Unique Values: 17



### Vehicle First Impact Location

In [15]:
# Vehicle First Impact Location: Replace NA values with a string "N/A"
display_unique_values(df, 'Vehicle First Impact Location')
df['Vehicle First Impact Location'] = df['Vehicle First Impact Location'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Vehicle First Impact Location')

Vehicle First Impact Location
TWELVE OCLOCK    66282
SIX OCLOCK       33563
ONE OCLOCK       15793
ELEVEN OCLOCK    13436
TWO OCLOCK        6050
TEN OCLOCK        5994
SEVEN OCLOCK      4179
FOUR OCLOCK       4003
FIVE OCLOCK       3968
EIGHT OCLOCK      3590
THREE OCLOCK      3280
NINE OCLOCK       3274
UNKNOWN           2981
UNDERSIDE          518
NON-COLLISION      385
ROOF TOP           347
Name: count, dtype: int64

Length of Unique Values: 16

Column Length: 167799
NA values:  156

__________________________

Vehicle First Impact Location
TWELVE OCLOCK    66282
SIX OCLOCK       33563
ONE OCLOCK       15793
ELEVEN OCLOCK    13436
TWO OCLOCK        6050
TEN OCLOCK        5994
SEVEN OCLOCK      4179
FOUR OCLOCK       4003
FIVE OCLOCK       3968
EIGHT OCLOCK      3590
THREE OCLOCK      3280
NINE OCLOCK       3274
UNKNOWN           2981
UNDERSIDE          518
NON-COLLISION      385
ROOF TOP           347
N/A                156
Name: count, dtype: int64

Length of Unique Values: 17

Co

### Vehicle Damage Extent

In [16]:
# Vehicle Damage Extent: Replace NA values with a string "N/A"
display_unique_values(df, 'Vehicle Damage Extent')
df['Vehicle Damage Extent'] = df['Vehicle Damage Extent'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Vehicle Damage Extent')

Vehicle Damage Extent
DISABLING      60767
FUNCTIONAL     43906
SUPERFICIAL    42845
DESTROYED       7371
UNKNOWN         6440
NO DAMAGE       6068
OTHER            100
Name: count, dtype: int64

Length of Unique Values: 7

Column Length: 167799
NA values:  302

__________________________

Vehicle Damage Extent
DISABLING      60767
FUNCTIONAL     43906
SUPERFICIAL    42845
DESTROYED       7371
UNKNOWN         6440
NO DAMAGE       6068
N/A              302
OTHER            100
Name: count, dtype: int64

Length of Unique Values: 8

Column Length: 167799
NA values:  0


### Circumstance

In [17]:
# Circumstance : Replace NA values with a string "N/A"
display_unique_values(df, 'Circumstance')
df['Circumstance'] = df['Circumstance'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Circumstance')

Circumstance
RAIN, SNOW, WET                                                                                                                   11856
N/A, WET                                                                                                                           8859
N/A, RAIN, SNOW                                                                                                                    1145
SLEET, HAIL, FREEZ. RAIN, WET                                                                                                      1033
N/A, VISION OBSTRUCTION (INCL. BLINDED BY SUN)                                                                                     1007
ICY OR SNOW-COVERED, RAIN, SNOW                                                                                                     978
ANIMAL, N/A                                                                                                                         679
ICY OR SNOW-COVERED, N/A           

### Non-Motorist Substance Abuse

In [18]:
# Non-Motorist Substance Abuse: Replace NA values with a string "N/A"
display_unique_values(df, 'Non-Motorist Substance Abuse')
df['Non-Motorist Substance Abuse'] = df['Non-Motorist Substance Abuse'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Non-Motorist Substance Abuse')

Non-Motorist Substance Abuse
NONE DETECTED                           3755
UNKNOWN                                  227
ALCOHOL PRESENT                          140
ALCOHOL CONTRIBUTED                       36
N/A, NONE DETECTED                        24
ILLEGAL DRUG PRESENT                       4
MEDICATION PRESENT                         4
ILLEGAL DRUG CONTRIBUTED                   2
NONE DETECTED, UNKNOWN                     2
COMBINATION CONTRIBUTED                    2
ALCOHOL CONTRIBUTED, ALCOHOL PRESENT       1
OTHER                                      1
COMBINED SUBSTANCE PRESENT                 1
N/A, UNKNOWN                               1
ALCOHOL PRESENT, NONE DETECTED             1
Name: count, dtype: int64

Length of Unique Values: 15

Column Length: 167799
NA values:  163598

__________________________

Non-Motorist Substance Abuse
N/A                                     163598
NONE DETECTED                             3755
UNKNOWN                                    227


### Driver Substance Abuse

In [19]:
# Driver Substance Abuse : Replace NA values with a string "N/A"
display_unique_values(df, 'Driver Substance Abuse')
df['Driver Substance Abuse'] = df['Driver Substance Abuse'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Driver Substance Abuse')

Driver Substance Abuse
NONE DETECTED                 119451
UNKNOWN                        11650
ALCOHOL PRESENT                 3991
ALCOHOL CONTRIBUTED             1404
ILLEGAL DRUG PRESENT             254
MEDICATION PRESENT               116
ILLEGAL DRUG CONTRIBUTED          99
COMBINED SUBSTANCE PRESENT        89
MEDICATION CONTRIBUTED            62
OTHER                             58
COMBINATION CONTRIBUTED           45
Name: count, dtype: int64

Length of Unique Values: 11

Column Length: 167799
NA values:  30580

__________________________

Driver Substance Abuse
NONE DETECTED                 119451
N/A                            30580
UNKNOWN                        11650
ALCOHOL PRESENT                 3991
ALCOHOL CONTRIBUTED             1404
ILLEGAL DRUG PRESENT             254
MEDICATION PRESENT               116
ILLEGAL DRUG CONTRIBUTED          99
COMBINED SUBSTANCE PRESENT        89
MEDICATION CONTRIBUTED            62
OTHER                             58
COMBINATION CON

### Traffic Control

In [20]:
#### Traffic Control: Replace NA values with a string "N/A"
display_unique_values(df, 'Traffic Control')
df['Traffic Control'] = df['Traffic Control'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Traffic Control')

Traffic Control
NO CONTROLS                66799
TRAFFIC SIGNAL             57424
STOP SIGN                  12162
FLASHING TRAFFIC SIGNAL     2076
OTHER                       1952
YIELD SIGN                  1698
UNKNOWN                      278
PERSON                       277
WARNING SIGN                 154
RAILWAY CROSSING DEVICE       42
SCHOOL ZONE SIGN DEVICE       16
Name: count, dtype: int64

Length of Unique Values: 11

Column Length: 167799
NA values:  24921

__________________________

Traffic Control
NO CONTROLS                66799
TRAFFIC SIGNAL             57424
N/A                        24921
STOP SIGN                  12162
FLASHING TRAFFIC SIGNAL     2076
OTHER                       1952
YIELD SIGN                  1698
UNKNOWN                      278
PERSON                       277
WARNING SIGN                 154
RAILWAY CROSSING DEVICE       42
SCHOOL ZONE SIGN DEVICE       16
Name: count, dtype: int64

Length of Unique Values: 12

Column Length: 167799
NA val

### Light

In [21]:
# Light: Replace NA values with a string "N/A"
display_unique_values(df, 'Light')
df['Light'] = df['Light'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Light')

Light
DAYLIGHT                    113822
DARK LIGHTS ON               38136
DARK NO LIGHTS                4779
DUSK                          3821
DAWN                          3316
DARK -- UNKNOWN LIGHTING      1529
UNKNOWN                        664
OTHER                          339
Name: count, dtype: int64

Length of Unique Values: 8

Column Length: 167799
NA values:  1393

__________________________

Light
DAYLIGHT                    113822
DARK LIGHTS ON               38136
DARK NO LIGHTS                4779
DUSK                          3821
DAWN                          3316
DARK -- UNKNOWN LIGHTING      1529
N/A                           1393
UNKNOWN                        664
OTHER                          339
Name: count, dtype: int64

Length of Unique Values: 9

Column Length: 167799
NA values:  0


### Surface Condition

In [22]:
# Surface Condition: Replace NA values with a string "N/A"
display_unique_values(df, 'Surface Condition')
df['Surface Condition'] = df['Surface Condition'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Surface Condition')

Surface Condition
DRY                       117504
WET                        27689
ICE                         1032
SNOW                         936
UNKNOWN                      504
SLUSH                        201
OTHER                        162
MUD, DIRT, GRAVEL             46
WATER(STANDING/MOVING)        39
OIL                           28
SAND                           4
Name: count, dtype: int64

Length of Unique Values: 11

Column Length: 167799
NA values:  19654

__________________________

Surface Condition
DRY                       117504
WET                        27689
N/A                        19654
ICE                         1032
SNOW                         936
UNKNOWN                      504
SLUSH                        201
OTHER                        162
MUD, DIRT, GRAVEL             46
WATER(STANDING/MOVING)        39
OIL                           28
SAND                           4
Name: count, dtype: int64

Length of Unique Values: 12

Column Length: 167799
NA

### Collision Type

In [23]:
# Collision Type: Replace NA values with a string "N/A"
display_unique_values(df, 'Collision Type')
df['Collision Type'] = df['Collision Type'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Collision Type')

Collision Type
SAME DIR REAR END               54542
STRAIGHT MOVEMENT ANGLE         29520
OTHER                           18512
SAME DIRECTION SIDESWIPE        15791
SINGLE VEHICLE                  15454
HEAD ON LEFT TURN               12628
SAME DIRECTION RIGHT TURN        3731
HEAD ON                          3661
SAME DIRECTION LEFT TURN         3607
OPPOSITE DIRECTION SIDESWIPE     2799
ANGLE MEETS LEFT TURN            1968
ANGLE MEETS RIGHT TURN           1182
SAME DIR REND LEFT TURN           723
SAME DIR REND RIGHT TURN          714
UNKNOWN                           700
SAME DIR BOTH LEFT TURN           694
ANGLE MEETS LEFT HEAD ON          692
OPPOSITE DIR BOTH LEFT TURN       310
Name: count, dtype: int64

Length of Unique Values: 18

Column Length: 167799
NA values:  571

__________________________

Collision Type
SAME DIR REAR END               54542
STRAIGHT MOVEMENT ANGLE         29520
OTHER                           18512
SAME DIRECTION SIDESWIPE        15791
SINGLE VEHI

### Weather

In [26]:
# Weather: Replace NA values with a string "N/A"
display_unique_values(df, 'Weather')
df['Weather'] = df['Weather'].fillna('N/A')
print("\n__________________________\n")
display_unique_values(df, 'Weather')

Weather
CLEAR                       113769
RAINING                      20079
CLOUDY                       16926
N/A                          13037
SNOW                          1432
UNKNOWN                        675
FOGGY                          639
WINTRY MIX                     388
OTHER                          357
SLEET                          213
SEVERE WINDS                   154
BLOWING SNOW                   115
BLOWING SAND, SOIL, DIRT        15
Name: count, dtype: int64

Length of Unique Values: 13

Column Length: 167799
NA values:  0

__________________________

Weather
CLEAR                       113769
RAINING                      20079
CLOUDY                       16926
N/A                          13037
SNOW                          1432
UNKNOWN                        675
FOGGY                          639
WINTRY MIX                     388
OTHER                          357
SLEET                          213
SEVERE WINDS                   154
BLOWING SNOW           

In [28]:
df.columns

Index(['Agency Name', 'ACRS Report Type', 'Crash Date/Time', 'Route Type',
       'Road Name', 'Cross-Street Type', 'Cross-Street Name',
       'Off-Road Description', 'Municipality', 'Related Non-Motorist',
       'Collision Type', 'Weather', 'Surface Condition', 'Light',
       'Traffic Control', 'Driver Substance Abuse',
       'Non-Motorist Substance Abuse', 'Driver At Fault', 'Injury Severity',
       'Circumstance', 'Driver Distracted By', 'Vehicle Damage Extent',
       'Vehicle First Impact Location', 'Vehicle Second Impact Location',
       'Vehicle Body Type', 'Vehicle Movement', 'Vehicle Continuing Dir',
       'Vehicle Going Dir', 'Vehicle Year', 'Vehicle Make', 'Vehicle Model',
       'Equipment Problems', 'Latitude', 'Longitude', 'Location'],
      dtype='object')

In [29]:
# Function that takes as input a dataframe and list of columns and replaces NA values with a string "N/A" in each of the specified columns of the dataframe
def replace_na(df, columns):
    for col in columns:
        df[col] = df[col].fillna('N/A')
    return df

sanitize_cols = ["Route Type", "Road Name",	"Cross-Street Type", "Cross-Street Name", "Off-Road Description",	"Municipality",	"Related Non-Motorist",	"Collision Type", "Vehicle Model"]
df = replace_na(df, sanitize_cols)


print_unique_values(df)
print("\n__________________________\n")
print(f"NaN in df: {df.isna().sum()}")

Agency Name
Montgomery County Police     134943
MONTGOMERY                    10659
Rockville Police Departme      9514
Gaithersburg Police Depar      7430
Takoma Park Police Depart      2877
Maryland-National Capital      1122
ROCKVILLE                       630
GAITHERSBURG                    390
TAKOMA                          157
MCPARK                           77
Name: count, dtype: int64
NA values:  0
Max freq:  134943
Min freq:  77
len:  10

__________________________

ACRS Report Type
Property Damage Crash    106538
Injury Crash              60841
Fatal Crash                 420
Name: count, dtype: int64
NA values:  0
Max freq:  106538
Min freq:  420
len:  3

__________________________

Crash Date/Time
2018-12-10 18:10:00    11
2020-06-09 18:53:00    10
2019-03-28 09:30:00    10
2017-03-03 06:00:00    10
2019-05-16 08:00:00     9
                       ..
2022-07-30 03:19:00     1
2019-12-17 18:04:00     1
2016-09-18 23:07:00     1
2016-07-03 11:47:00     1
2022-12-15 09:00:00

## OUTPUT

In [30]:
df.to_csv('data/crash_reporting_drivers_data_sanitized.csv', index=False)

In [31]:
# Find the earliest and latest crash dates
print(f"Earliest Crash Date: {min(df['Crash Date/Time'])}")
print(f"Latest Crash Date: {max(df['Crash Date/Time'])}")

#display_unique_values(df, 'Road Name')

Earliest Crash Date: 2015-01-01 00:30:00
Latest Crash Date: 2023-10-24 20:29:00
